In [1]:
import gensim
from gensim.models.doc2vec import TaggedLineDocument
import numpy as np
import pandas as pd
from sklearn import svm


In [ ]:
#Version=2.0
#Using trainData to get doc2vec model
#Using doc2vec model to inferrence vector of testData

In [2]:
# train model Doc2Vec
#train vector:1600000
#test vector:359
#documents = TaggedLineDocument("../word2vce/trainAndTest")
#model = gensim.models.Doc2Vec(documents, size=100, window=5, min_count=5, workers=4, dm_mean=1)
##Persist a model to disk with:
#model.save("doc2vec.model")

In [2]:
#count number of words in vocabulary from word2vec in a sentence
#Input: a sentence 
#Output: number
def countWordInVocab(sentence):
    num = 0
    vocab = word2vec_model.vocab
    for word in sentence:
        if word in vocab:
            num += 1
        else:
            pass
    return num

#count number of neg and pos to be deleted
#Input: sentences
#Output: neg_number, pos_number
def cntNumNegPos(sentences):
    neg_deleted = 0
    pos_deleted = 0
    test_deleted = 0
    for index, sen in enumerate(sentences):
        if countWordInVocab(sen) == 0:
            if index < 800000:
                neg_deleted += 1
            elif index < 1600000:
                pos_deleted += 1
            else:
                test_deleted += 1
        else:
            pass
    return 800000-neg_deleted, 800000-pos_deleted, 359-test_deleted

#delete blank sentence
#Input:sentences
#Output: processed sentences
#parameter must be mutable, then can be changed in function
def deleteBlankSen(sentences):
    return [sen for sen in sentences if countWordInVocab(sen) > 0]

In [3]:
X_train = []
y_train = []
X_test = []
y_test = []

def prepareData():
    global X_train, y_train, X_test, y_test
    
    sentences = gensim.models.word2vec.LineSentence("../word2vce/train_text-norm")
    neg_num, pos_num, test_num = cntNumNegPos(sentences)
    print(neg_num, pos_num, test_num)
    doc2vecCorpus = deleteBlankSen(sentences)
    
    #把单个句子的每个单词用空格连起来，方便写入文件
    finalCorpus = [ " ".join(sen)  for sen in doc2vecCorpus]
    with open("doc2vecCorpus.txt", 'w') as outfile:
        outfile.write("\n".join(finalCorpus))
    
    documents = TaggedLineDocument("doc2vecCorpus.txt")
    doc2vec_model = gensim.models.Doc2Vec(documents, size=300, window=8, min_count=5, 
                                          workers=4, dm_mean=1, )
    ##Persist a model to disk with:
    doc2vec_model.save("doc2vec.model")
    #model = gensim.models.Doc2Vec.load("doc2vec.model")  # you can continue training with the loaded model!
    train_index = [index for index in range(neg_num+pos_num)]
    X_train = doc2vec_model.docvecs[train_index]
    y_train = [0] * neg_num + [4] * pos_num
    #test_index = [ index+neg_num+pos_num for index in range(test_num)]
    #X_test = doc2vec_model.docvecs[test_index]

    #testData = pd.read_csv("../word2vce/testData", encoding="latin-1", names=["label","text"])
    #test_sentences = gensim.models.word2vec.LineSentence("../word2vce/test_text-norm")
    #y_test = list(testData['label'])

In [12]:
word2vec_model = gensim.models.Word2Vec.load("word2vec.model") 
prepareData()

798531 798396 359


In [4]:
model = gensim.models.Doc2Vec.load("doc2vec.model")


In [5]:
test_sentences = gensim.models.word2vec.LineSentence("../word2vce/test_text-norm")

In [6]:
for sen in test_sentences:
    X_test.append(model.infer_vector(sen, alpha=0.1, min_alpha=0.0001, steps=5))


In [7]:
testData = pd.read_csv("../word2vce/testData", encoding="latin-1", names=["label","text"])
y_test = list(testData['label'])

In [10]:

# train SVM 
def train():
    global X_train, y_train, X_test, y_test
    #clf = svm.SVC(kernel='linear')
    clf = svm.LinearSVC(dual=False)
    clf.fit(X_train, y_train)
    #print("number of support vector: %d\t%d" %(clf.n_support_[0], clf.n_support_[1]))
    return clf
    #print SVM information
    #clf.n_support_
    #clf.dual_coef_
    #clf.coef_[0]
    
def compute_accuracy(y, y_):
    result = np.equal(y, y_).astype(int)
    return result.mean()



def main():
    global y_test
    prepareData()
    #train the SVM model
    clf = train()
    #predict
    y = clf.predict(X_test)
    acc = compute_accuracy(y, y_test)
    print("accuracy:%f" % (acc))

In [ ]:
clf = train()

In [ ]:
clf

In [ ]:
x=3

In [ ]:
if __name__ == "__main__":
    word2vec_model = gensim.models.Word2Vec.load("word2vec.model") 
    main()

798531 798396 359


Word2Vec settings: size=100, window=5, min_count=5, workers=4, alpha=0.025, window=5, min_count=5， sample=0.001, min_alpha=0.0001, sg=0, hs=0, negative=5, cbow_mean=1, 
iter=5, batch_words=10000
| Model                  | Accuracy      | model settings  |
| ---------------------- |-------------  | -----|
| AvgWord2vecSVM         | 74.0947       | LinearSVC(dual=False)|
| AvgWord2vecSVM         | 74.3733       | LinearSVC(dual=False) delete blank 3073 sentence|
| ClusterWord2VecSVM     | 66.0167       |   LinearSVC(dual=False)  k=200 |
|  NBSVM                 | 77.1588       | liblinear -s 2 n-gram=1|
| NBSVM                  | 82.4513       | liblinear -s 2 n-gram=2|
| NBSVM                  | 84.1226       | liblinear -s 2 n-gram=3|
| NBSVM                  | 83.8440       | liblinear -s 2 n-gram=3|
| Para2Vec               | 52.0891       | size=100, window=5, min_count=5, dm_mean=0|
| Para2Vec               | 51.2535       | size=100, window=5, min_count=5, dm_mean=1|
| Para2Vec               | 50.6964       | size=300, window=5, min_count=5, dm_mean=1|

Word2Vec settings: size=1000, window=5, min_count=5, workers=4, alpha=0.025, window=5,
    min_count=5， sample=0.00001, min_alpha=0.0001, sg=1, hs=0, negative=5, cbow_mean=1, 
    iter=5,batch_words=10000


| Model                  | Accuracy      | model settings  |
| ---------------------- |-------------  | -----|
| AvgWord2vecSVM         | 70.4735       | LinearSVC(dual=False) C=0.03 loss='squared_hinge' penalty='L2'|


| AvgWord2vecSVM         | 74.0947       | LinearSVC(dual=False) C=0.03 loss='squared_hinge' penalty='L2'|
| AvgWord2vecSVM         | 74.3733       | LinearSVC(dual=False) delete blank 3073 sentence|
| ClusterWord2VecSVM     | 66.0167       |   LinearSVC(dual=False)  k=200 |
|  NBSVM                 | 77.1588       | liblinear -s 2 n-gram=1|
| NBSVM                  | 82.4513       | liblinear -s 2 n-gram=2|
| NBSVM                  | 84.1226       | liblinear -s 2 n-gram=3|
| NBSVM                  | 83.8440       | liblinear -s 2 n-gram=3|
| Para2Vec               | 52.0891       | size=100, window=5, min_count=5, dm_mean=0|
| Para2Vec               | 51.2535       | size=100, window=5, min_count=5, dm_mean=1|
| Para2Vec               | 50.6964       | size=300, window=5, min_count=5, dm_mean=1|

In [1]:
798531 + 798396 + 359

1597286